# Import

In [1]:
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from load_data.prepared_custom_ds import CustomDataset
from utilities.config_load import load_config

import time
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

from sklearn.model_selection import train_test_split
import os.path as osp
import pandas as pd
import os
from tqdm import tqdm

In [2]:
import gc
torch.cuda.empty_cache()
gc.collect()

31

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["PYTORCH_USE_CUDA_DSA"] = "1"

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load data and config

In [4]:
CONFIG_PATH = "configs/"
config = load_config(CONFIG_PATH, "config.yaml")

In [5]:
config

{'dataset': {'train_img_path': 'dataset/train_v2',
  'test_img_path': 'dataset/test_v2',
  'mask_path': 'dataset/train_masks',
  'reshaped_img_path': 'dataset/reshaped_img',
  'dir_path': 'dataset'},
 'original_img_size': 768,
 'new_img_size': 256,
 'project_path': 'C:/Users/da4nik/Segmentation',
 'model': {'svs_path': 'model_svs/'}}

In [6]:
labels = list(pd.read_csv(osp.join(config['dataset']['dir_path'], 
                                                'train_ship_segmentations_v2.csv'))["EncodedPixels"].fillna('').str.split())
img_ids = list(pd.read_csv(osp.join(config['dataset']['dir_path'], 
                                                'train_ship_segmentations_v2.csv'))["ImageId"])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(img_ids, 
                                                    labels, 
                                                    test_size=0.1,
                                                    random_state=42)

# Model

In [8]:
def ConvBlock(first_chanels, second_chanels, kernel_size, dropout_rate):
    return nn.Sequential(
        nn.BatchNorm2d(first_chanels),
        nn.Conv2d(first_chanels, second_chanels, kernel_size, padding='same'),
        nn.ReLU(inplace=True),
        nn.Dropout(dropout_rate),
        nn.BatchNorm2d(second_chanels),
        nn.Conv2d(second_chanels, second_chanels, kernel_size, padding='same'),
        nn.ReLU(inplace=True)
    )

In [9]:
class Unet_Encoder(nn.Module):
    def __init__(self, kernel_size, dropout_rate, nkernels):
        super(Unet_Encoder, self).__init__()
        self.kernel_size = kernel_size
        self.dropout_rate = dropout_rate
        self.nkernels = nkernels
        self.conv1 = ConvBlock(3, nkernels, self.kernel_size, self.dropout_rate)
        self.conv2 = ConvBlock(nkernels, nkernels*2, self.kernel_size, self.dropout_rate)
        self.conv3 = ConvBlock(nkernels*2, nkernels*4, self.kernel_size, self.dropout_rate)
        self.conv4 = ConvBlock(nkernels*4, nkernels*8, self.kernel_size, self.dropout_rate)
        self.maxpool_list = nn.ModuleList([nn.MaxPool2d(kernel_size=2) for _ in range(4)])
        self.conv_list = nn.ModuleList([self.conv1, self.conv2, self.conv3, self.conv4])

    '''def init_weights(self):
        for module in self.modules():
            if isinstance(module, (nn.Linear, nn.Conv2d)):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0.01)'''

    def forward(self, input):
        list_skips = list()
        for i in range(4):
            skip = self.conv_list[i](input)
            input = self.maxpool_list[i](skip)
            list_skips.append(skip)
        return input, list_skips

In [10]:
class Unet_Decoder(nn.Module):
    def __init__(self, kernel_size, dropout_rate, nkernels):
        super(Unet_Decoder, self).__init__()
        self.kernel_size = kernel_size
        self.dropout_rate = dropout_rate
        self.nkernels = nkernels
        self.conv5 = ConvBlock(nkernels*8, nkernels*16, self.kernel_size, self.dropout_rate)
        self.conv6 = ConvBlock(nkernels*16, nkernels*8, self.kernel_size, self.dropout_rate)
        self.conv7 = ConvBlock(nkernels*8, nkernels*4, self.kernel_size, self.dropout_rate)
        self.conv8 = ConvBlock(nkernels*4, nkernels*2, self.kernel_size, self.dropout_rate)
        self.conv_list = nn.ModuleList([self.conv5, self.conv6, self.conv7, self.conv8])
        self.convt_list = nn.ModuleList([nn.ConvTranspose2d(nkernels*(2**(4-i)), nkernels*((2**(4-i))//2), kernel_size=(2, 2), stride=(2, 2)) 
                                           for i in range(4)])

    
    '''def init_weights(self):
        for module in self.modules():
            if isinstance(module, (nn.Linear, nn.Conv2d, nn.ConvTranspose2d)):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0.01)'''

    
    def forward(self, input, list_skips):
        for i in range(4):
            if i==0:
                out = self.conv_list[i](input)
                out = self.convt_list[i](out)
            else:
                out = self.conv_list[i](torch.cat((out, list_skips[4-i]), 1)) # channel
                out = self.convt_list[i](out)
        return out

In [11]:
class Model_Unet(nn.Module):
    def __init__(self, kernel_size, dropout_rate, nkernels, output_chanels):
        super(Model_Unet, self).__init__()
        self.output_chanels = output_chanels
        self.kernel_size = kernel_size
        self.dropout_rate = dropout_rate
        self.nkernels = nkernels
        self.enc_layer = Unet_Encoder(self.kernel_size, self.dropout_rate, self.nkernels)
        self.dec_layer = Unet_Decoder(self.kernel_size, self.dropout_rate, self.nkernels)
        self.conv9 = ConvBlock(self.nkernels*2, self.nkernels, self.kernel_size, self.dropout_rate)
        self.conv10 = nn.Conv2d(self.nkernels, self.output_chanels, (1, 1), padding='same')
        self.relu = nn.ReLU()
        self.activation = nn.Sigmoid()
        

    '''def init_weights(self):
        for module in self.modules():
            if isinstance(module, (nn.Linear, nn.Conv2d)):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0.01)'''

    
    def forward(self, input):
        out, list_skips = self.enc_layer(input)
        out = self.dec_layer(out, list_skips)
        out = self.conv9(torch.cat((out, list_skips[0]), 1)) # channel concat
        out = self.relu(out)
        out = self.conv10(out)
        out = self.activation(out)
        return out

# Train function

In [12]:
def train_step(model, loss_fn, opt, loader):
    loss_per_batches = 0
    elapsed = 0
    start_epoch2 = time.time()
    for i, data in tqdm(enumerate(loader), total=231000//160):

        start_epoch = time.time()
        features, labels = data
        features, labels = features.to(device), labels.to(device)
        opt.zero_grad()
        
        y_pred = model(features)
        loss = loss_fn(y_pred, labels)
        loss.backward()
        
        opt.step()
        
        loss_per_batches += loss
        
        end_epoch = time.time()
        elapsed += (end_epoch - start_epoch)
        
    print("train = " + str(elapsed))
    print("train + load = " + str(time.time() - start_epoch2))
    return loss_per_batches/(i+1)

In [13]:
def train(model, loss_fn, opt, train_loader, val_loader, save_treshold=25, epochs=50, model_name='model_name'):
        
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    writer = SummaryWriter('runs/' + model_name + '_{}'.format(timestamp))
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', patience=3, verbose=True)
    
    for epoch in range(epochs):
        start_epoch = time.time()
        print('EPOCH {}:'.format(epoch + 1))
        
        model.train()
        avg_loss = train_step(model, loss_fn, opt, train_loader)
        model.eval()

        vloss = 0
        counter = 0
        with torch.inference_mode():
            for i, vdata in enumerate(val_loader):
                vfeatures, vlabels = vdata
                vfeatures, vlabels = vfeatures.to(device), vlabels.to(device)

                y_pred = model(vfeatures)
                vloss += loss_fn(y_pred, vlabels)
                counter = i

        avg_vloss = vloss / (counter + 1)
        
        scheduler.step(avg_loss)
        
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
        
        writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch + 1)
        
        if (epoch + 1) % save_treshold == 0:
            model_path = config['model']['svs_path'] + model_name +'_{}_{}'.format(timestamp, (epoch + 1))
            torch.save(model.state_dict(), model_path)
        end_epoch = time.time()
        elapsed = end_epoch - start_epoch
        print("Time per epoch {}s".format(elapsed))

# Creating dataloader and model objects

In [14]:
dataset = CustomDataset(config, X_train, y_train)
vdataset = CustomDataset(config, X_test, y_test)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=160, num_workers=0, shuffle=False)
vdataloader = torch.utils.data.DataLoader(vdataset, batch_size=160, num_workers=0, shuffle=False)

In [15]:
model = Model_Unet(kernel_size=3, dropout_rate=0.15, nkernels=2, output_chanels=1)
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
model.to(device)
pass

In [16]:
summary(model)
pass

Layer (type:depth-idx)                   Param #
├─Unet_Encoder: 1-1                      --
|    └─Sequential: 2-1                   --
|    |    └─BatchNorm2d: 3-1             6
|    |    └─Conv2d: 3-2                  56
|    |    └─ReLU: 3-3                    --
|    |    └─Dropout: 3-4                 --
|    |    └─BatchNorm2d: 3-5             4
|    |    └─Conv2d: 3-6                  38
|    |    └─ReLU: 3-7                    --
|    └─Sequential: 2-2                   --
|    |    └─BatchNorm2d: 3-8             4
|    |    └─Conv2d: 3-9                  76
|    |    └─ReLU: 3-10                   --
|    |    └─Dropout: 3-11                --
|    |    └─BatchNorm2d: 3-12            8
|    |    └─Conv2d: 3-13                 148
|    |    └─ReLU: 3-14                   --
|    └─Sequential: 2-3                   --
|    |    └─BatchNorm2d: 3-15            8
|    |    └─Conv2d: 3-16                 296
|    |    └─ReLU: 3-17                   --
|    |    └─Dropout: 3-18     

# Training

In [17]:
train(model, loss_fn, optimizer, dataloader, vdataloader, 50, epochs=100, model_name='first_try')

EPOCH 1:


  0%|                                                                                         | 0/1443 [00:00<?, ?it/s]C:\Users\da4nik\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
 90%|██████████████████████████████████████████████████████████████████████▍       | 1304/1443 [13:46<01:28,  1.58it/s]


train = 147.50356078147888
train + load = 826.4572033882141
LOSS train 0.0908844918012619 valid 0.005149719305336475
Time per epoch 910.5090363025665s
EPOCH 2:


 90%|██████████████████████████████████████████████████████████████████████▍       | 1304/1443 [13:33<01:26,  1.60it/s]


train = 108.46459126472473
train + load = 813.9887747764587
LOSS train 0.004183068871498108 valid 0.004187147133052349
Time per epoch 897.270733833313s
EPOCH 3:


 90%|██████████████████████████████████████████████████████████████████████▍       | 1304/1443 [13:43<01:27,  1.58it/s]


train = 128.67351508140564
train + load = 823.1553995609283
LOSS train 0.003634538734331727 valid 0.0035843043588101864
Time per epoch 905.8035776615143s
EPOCH 4:


  8%|██████▎                                                                        | 115/1443 [01:13<14:05,  1.57it/s]


KeyboardInterrupt: 

In [13]:
device

device(type='cuda', index=0)

# Test

In [9]:
start = time.time()
for data in tqdm(dataloader):
    features, labels = data
    try:
        features, labels = features.to(device), labels.to(device)
    except:
        print(features)
        print(str(features.dtype) + " -> " + str(features.shape))

100%|████████████████████████████████████████████████████████████████████████████| 17380/17380 [06:27<00:00, 44.88it/s]


In [ ]:
res = model(torch.zeros(16, 3, 256, 256).to(device))

In [16]:
torch.zeros(16, 3, 256, 256).dtype

torch.float32

In [ ]:
res.shape

In [ ]:
vec1 = torch.zeros(16, 1, 200, 200).to(device)

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["PYTORCH_USE_CUDA_DSA"] = "1"

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
vec2 = torch.ones(16, 1, 200, 200)

In [10]:
loss_fun = torch.nn.BCELoss()

In [11]:
res = loss_fun(torch.ones(16, 1, 200, 200), torch.rand(16, 1, 200, 200))

In [12]:
res

tensor(49.9320)

In [ ]:
for i in range(4):
    print(str(2**(4-i)) + " => " + str((2**(4-i))//2))

In [ ]:
max = nn.MaxPool2d(2)

In [ ]:
ct = nn.ConvTranspose2d(16, 16//2, kernel_size=(2, 2), stride=(2, 2))

In [ ]:
res = max(torch.zeros(16, 200, 200))

In [ ]:
res.shape

In [ ]:
test = ct(torch.zeros(16, 200, 200))

In [ ]:
test.shape

In [ ]:
print(1)

In [ ]:
for pos, i in enumerate(dataloader):
    print(i[0].shape)